In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/DS'

/content/drive/MyDrive/DS


In [3]:
!pip install requests_cache

     |████████████████████████████████| 636 kB 8.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.0.1 which is incompatible.


In [4]:
from bs4 import BeautifulSoup
import requests as req
import requests_cache
import time
import json
import numpy as np
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import itertools
import pandas as pd
from random import *

In [5]:
requests_cache.install_cache()

In [ ]:
#requests to get top250 films
url = 'https://imdb-api.com/en/API/Top250Movies/k_p01dgqso'
r=req.get(url)
data=r.text
r.from_cache

True

In [ ]:
#get film's ids from json (api)
data = json.loads(data)
data = data['items']
id_ = []
for item in data:
  id_.append(item['id'])

In [ ]:
# read file id_ to get bottom film's ids
lst_id = []
with open('id_.txt') as f:
  for l in f:
    lst_id.append(l.split('\n')[0])

50


In [ ]:
for i in range(50):
  temp = randint(0,249)
  while id_[temp] in lst_id:
    temp = randint(0,249)
  lst_id.append(id_[temp])

In [ ]:
reviews = []

In [ ]:
# collect reviews from list of id
for id in lst_id:
  url = f'https://imdb-api.com/en/API/Reviews/k_p01dgqso/{id}'
  review = req.get(url)
  if review.from_cache:
    print('Get data from cache')
  else:
    time.sleep(1)
  review = json.loads(review.text)
  reviews.extend(review['items'])

Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache


In [ ]:
store = reviews

In [ ]:
titles, contents , rates = [],[],[]

In [ ]:
for r in reviews:
  titles.append(r['title'])
  contents.append(r['content'])
  rates.append(r['rate'])

df = pd.DataFrame(columns = ['Title','Content','Rate'])
df.Title = titles
df.Content = contents
df.Rate = rates
df.head(5)

,Title,Content,Rate
0,Good light hearted fun.,This is nowhere near as bad as some of the rev...,7
1,Wartime Themes,Love and sacrifice during WWII underlie the st...,
2,"""Critics"" gave Gravity a 9, and flunked Area 51.","As a big sci-fi fan and UFO-enthusiast, I real...",9
3,A slow start builds to a riveting climax,At the beginning of this film I thought we wer...,
4,"If God is a movie, he's this one.",My moron friend John says that this movie is t...,9


In [ ]:
df.Rate.value_counts()

10    785
1     449
      263
9     239
8     173
2     133
7     110
3      88
6      84
4      65
5      62
Name: Rate, dtype: int64

In [ ]:
df.to_excel('./data.xlsx',index=False)

In [9]:
temp_df = pd.read_excel('./data.xlsx')

In [7]:
temp_df.dropna()

,Title,Content,Rate
0,Good light hearted fun.,This is nowhere near as bad as some of the rev...,7.0
2,"""Critics"" gave Gravity a 9, and flunked Area 51.","As a big sci-fi fan and UFO-enthusiast, I real...",9.0
4,"If God is a movie, he's this one.",My moron friend John says that this movie is t...,9.0
5,Carrie On to See a Different Film,The popular TV series Sex and the City has ran...,3.0
6,Oh my god,"Awful,disgusting,pointless I can write more ne...",1.0
...,...,...,...
2445,It's all in the ending,Cole (Haley Joel Osment) is a young boy troubl...,10.0
2447,This film is amazing,"Robocop 1 and 2 were great, and this movie was...",10.0
2448,Simultaneously personal and panoramic,"Sweeping, epic and literate version of British...",10.0
2449,Tit-le,Very funny I know - almost as funny as the mov...,3.0


In [10]:
temp_df.Rate.dropna()

0        7.0
2        9.0
4        9.0
5        3.0
6        1.0
        ... 
2446     4.0
2447    10.0
2448    10.0
2449     3.0
2450     9.0
Name: Rate, Length: 2188, dtype: float64

In [11]:
temp_df.Content.dropna()

0       This is nowhere near as bad as some of the rev...
1       Love and sacrifice during WWII underlie the st...
2       As a big sci-fi fan and UFO-enthusiast, I real...
3       At the beginning of this film I thought we wer...
4       My moron friend John says that this movie is t...
                              ...                        
2445    Cole (Haley Joel Osment) is a young boy troubl...
2447    Robocop 1 and 2 were great, and this movie was...
2448    Sweeping, epic and literate version of British...
2449    Very funny I know - almost as funny as the mov...
2450    Once again. What is wrong with everybody?! I d...
Name: Content, Length: 2240, dtype: object

In [12]:
temp_df.Title.dropna()

0                                Good light hearted fun.
1                                         Wartime Themes
2       "Critics" gave Gravity a 9, and flunked Area 51.
3               A slow start builds to a riveting climax
4                      If God is a movie, he's this one.
                              ...                       
2446               Way Too Long and Way Too Conventional
2447                                This film is amazing
2448               Simultaneously personal and panoramic
2449                                              Tit-le
2450         Great Movie - What is Wrong with Everybody!
Name: Title, Length: 2451, dtype: object

In [8]:
temp_df.Rate.value_counts()

10.0    785
1.0     449
9.0     239
8.0     173
2.0     133
7.0     110
3.0      88
6.0      84
4.0      65
5.0      62
Name: Rate, dtype: int64

In [13]:
temp_df.tail(10)

,Title,Content,Rate
2441,Kurosawa at his best and most subtle,This is one of those rare movies I had to watc...,10.0
2442,This is a movie found from the garbage landfill.,This is movie that should have never been made...,1.0
2443,Intense and Highly Entertaining,"""High and Low"" could be considered two movies....",10.0
2444,"WTF, man?",Dude. This movie was AWESOME. I loved when he ...,10.0
2445,It's all in the ending,Cole (Haley Joel Osment) is a young boy troubl...,10.0
2446,Way Too Long and Way Too Conventional,NaN,4.0
2447,This film is amazing,"Robocop 1 and 2 were great, and this movie was...",10.0
2448,Simultaneously personal and panoramic,"Sweeping, epic and literate version of British...",10.0
2449,Tit-le,Very funny I know - almost as funny as the mov...,3.0
2450,Great Movie - What is Wrong with Everybody!,Once again. What is wrong with everybody?! I d...,9.0
